In [4]:
%matplotlib


Using matplotlib backend: MacOSX


In [15]:
import os
import pandas as pd
import numpy as np
import matplotlib.dates as dates
import matplotlib.pyplot as plt

In [6]:
print os.getcwd()
os.chdir("/Users/soltesz/src/github.com/stephen-soltesz/ipython-testing")

/Users/soltesz/src/github.com/stephen-soltesz/ipython-testing


In [11]:
# generate data with:
# cat daily_discards.sql | bq query --format=csv --max_rows=1000000 --nouse_legacy_sql > out.csv
df = pd.read_csv('daily_discards.csv')

In [12]:
len(df)

86688

In [13]:
%matplotlib auto

Using matplotlib backend: MacOSX


In [16]:
sites = [
    ['dfw', 'lga', 'iad'],
    ['sea', 'atl', 'den'],
    ['mia', 'nuq', 'ord'],
]

fig, axes = plt.subplots(nrows=3, ncols=3)
for i, site_row in enumerate(sites):
    for j, site in enumerate(site_row):
        axes[i, j].set_title(site)
        if j != 0:
            axes[i, j].set_yticklabels([])
        if i != len(sites)-1:
            axes[i, j].set_xticklabels([])
        for h in set(df['hostname']):
            if 'mlab1.' + site in h:
                axes[i, j].plot_date(dates.epoch2num(df['ts'][ df['hostname'] == h]), df['total_discards'][ df['hostname'] == h], ls='-', ms=0, label=h[:11])
        axes[i, j].set_ylim(1, 1000000)
        axes[i, j].tick_params(axis='x', labelrotation=90)
        axes[i, j].grid(color='#dddddd')
        axes[i, j].legend(loc=4, fontsize='x-small')
        axes[i, j].semilogy()
        
fig.subplots_adjust(hspace=0.3, wspace=0.4)



In [17]:
len(df['hostname'])

86688

In [18]:
#    plt.plot(df['total_discards'][ df['hostname'] == h])

In [19]:
df = pd.read_csv('sidestream-trial-6w.csv')

In [22]:
print len(df), max(df['rate_mbps'])

hosts = [
    ['mlab1.lga03', 'mlab1.lga04'],
    ['mlab1.dfw02', 'mlab1.dfw03'],
]

fig, axes = plt.subplots(nrows=2, ncols=2)

for i, host_row in enumerate(hosts):
    for j, host in enumerate(host_row):
        for period in ['0w', '1w', '2w']: # , '3w', '4w', '5w']:
            ds = df[ (df['period'] == period) & (df['hostname'] == host) ]
            n, bins, patches = axes[i, j].hist(ds['rate_mbps'], len(ds['rate_mbps']),
                               histtype='step', normed=1, cumulative=True, label='cdf-' + period,
                               ls='-')
        # axes[i].set_aspect('equal')
        axes[i, j].set_axisbelow(True)
        axes[i, j].semilogx()
        axes[i, j].legend(loc=2)
        axes[i, j].grid(color='#dddddd')
        axes[i, j].set_title(host)
    
plt.show()
print n, len(bins)

262916 901.275707813
[0.03767123 0.05950342 0.07719749 ... 0.99985731 0.99985731 1.        ] 7009


In [23]:
df = pd.read_csv('/Users/soltesz/Downloads/switch-dfw02-pct.csv')
plt.plot_date(dates.epoch2num(df['ts']), df['total_discards'], ls='-', ms=0)
plt.tick_params(axis='x', labelrotation=90)
plt.show()